In [ ]:
!wget https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip

--2024-08-30 12:57:31--  https://archive.ics.uci.edu/static/public/275/bike+sharing+dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bike+sharing+dataset.zip.1’

bike+sharing+datase     [  <=>               ] 273.43K   912KB/s    in 0.3s    

2024-08-30 12:57:32 (912 KB/s) - ‘bike+sharing+dataset.zip.1’ saved [279992]



In [ ]:
# Update scikit-learn to the latest version
!pip install --upgrade scikit-learn

# Verify the update
import sklearn
print("scikit-learn version:", sklearn.__version__)


scikit-learn version: 1.5.1


In [ ]:
!unzip bike+sharing+dataset.zip

Archive:  bike+sharing+dataset.zip
replace Readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Readme.txt              
replace day.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: day.csv                 
replace hour.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hour.csv                


In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd

In [ ]:
df= pd.read_csv('hour.csv')


In [ ]:
print(df)

       instant      dteday  season  yr  mnth  hr  holiday  weekday  \
0            1  2011-01-01       1   0     1   0        0        6   
1            2  2011-01-01       1   0     1   1        0        6   
2            3  2011-01-01       1   0     1   2        0        6   
3            4  2011-01-01       1   0     1   3        0        6   
4            5  2011-01-01       1   0     1   4        0        6   
...        ...         ...     ...  ..   ...  ..      ...      ...   
17374    17375  2012-12-31       1   1    12  19        0        1   
17375    17376  2012-12-31       1   1    12  20        0        1   
17376    17377  2012-12-31       1   1    12  21        0        1   
17377    17378  2012-12-31       1   1    12  22        0        1   
17378    17379  2012-12-31       1   1    12  23        0        1   

       workingday  weathersit  temp   atemp   hum  windspeed  casual  \
0               0           1  0.24  0.2879  0.81     0.0000       3   
1              

In [ ]:

df['day_night'] = df['hr'].apply(lambda x: 'day' if 6 <= x <= 18 else 'night')
df.drop(['instant', 'casual', 'registered'], axis=1, inplace=True)
df['dteday'] = pd.to_datetime(df.dteday)
df['season'] = df.season.astype('category')
df['holiday'] = df.holiday.astype('category')
df['weekday'] = df.weekday.astype('category')
df['weathersit'] = df.weathersit.astype('category')
df['workingday'] = df.workingday.astype('category')
df['mnth'] = df.mnth.astype('category')
df['yr'] = df.yr.astype('category')
df['hr'] = df.hr.astype('category')
df.drop(columns=['dteday'], inplace=True)

#storing the target variable separately
X = df.drop(columns=['cnt']) #Features
y = df['cnt'] #Target

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   season      17379 non-null  category
 1   yr          17379 non-null  category
 2   mnth        17379 non-null  category
 3   hr          17379 non-null  category
 4   holiday     17379 non-null  category
 5   weekday     17379 non-null  category
 6   workingday  17379 non-null  category
 7   weathersit  17379 non-null  category
 8   temp        17379 non-null  float64 
 9   atemp       17379 non-null  float64 
 10  hum         17379 non-null  float64 
 11  windspeed   17379 non-null  float64 
 12  cnt         17379 non-null  int64   
 13  day_night   17379 non-null  object  
dtypes: category(8), float64(4), int64(1), object(1)
memory usage: 952.8+ KB


# **Creating various feature interactions**

In [ ]:
#creation of interaction features
X['temp_hum'] = X['temp'] * X['hum'] #num
X['temp_windspeed'] = X['temp'] * X['windspeed'] #num
X['hum_windspeed'] = X['hum'] * X['windspeed']  #num
X['weekd-workd']=X['weekday'].cat.codes*X['workingday'].cat.codes  #cat
X['holid-workd']=X['holiday'].cat.codes*X['workingday'].cat.codes  #cat
X['season_temp']=X['season'].cat.codes*X['temp'] #mixed


## Data inconsistencies and Missing Values Check

In [ ]:
df.isnull().sum()
print(df.isnull().sum())
print(df.dtypes)

season        0
yr            0
mnth          0
hr            0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
cnt           0
day_night     0
dtype: int64
season        category
yr            category
mnth          category
hr            category
holiday       category
weekday       category
workingday    category
weathersit    category
temp           float64
atemp          float64
hum            float64
windspeed      float64
cnt              int64
day_night       object
dtype: object


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
#from sklearn.preprocessing import TargetEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from category_encoders import TargetEncoder


## **Feature Engineering and Model Setup**

In [ ]:
#model1 with only num interactions
#Num features
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'temp_hum', 'temp_windspeed']
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


#Cat features
categorical_features_onehot = ['season', 'weathersit', 'day_night']
categorical_pipeline_onehot = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder())  #Target encoder
])

categorical_features_target = ['season', 'weathersit', 'day_night']
categorical_pipeline_target = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('target', TargetEncoder())  #Target encoder
])



In [ ]:
print(categorical_pipeline_onehot)
categorical_pipeline_onehot

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder())])


Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('onehot', OneHotEncoder())])

In [ ]:
print(categorical_pipeline_target)
categorical_pipeline_target

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('target', TargetEncoder())])


Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('target', TargetEncoder())])

In [ ]:
numerical_pipeline

Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())])

In [ ]:
# X[numerical_features] = numerical_pipeline.fit_transform(X[numerical_features])
# X[numerical_features]

In [ ]:
#X_encoded_onehot = categorical_pipeline_onehot.fit_transform(X[categorical_features_onehot], y)

In [ ]:
#X_encoded_target = categorical_pipeline_target.fit_transform(X[categorical_features_target], y)

In [ ]:
# Column Transformer with OneHotEncoding
preprocessor_onehot = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline_onehot, categorical_features_onehot)
    ])

preprocessor_onehot

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['temp', 'atemp', 'hum', 'windspeed',
                                  'temp_hum', 'temp_windspeed']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot', OneHotEncoder())]),
                                 ['season', 'weathersit', 'day_night'])])

In [ ]:
# Column Transformer with TargetEncoding
preprocessor_target = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline_target, categorical_features_target)
    ])

preprocessor_target

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['temp', 'atemp', 'hum', 'windspeed',
                                  'temp_hum', 'temp_windspeed']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('target', TargetEncoder())]),
                                 ['season', 'weathersit', 'day_night'])])

In [ ]:
LiReg_model_onehot = Pipeline(steps=[('preprocessor', preprocessor_onehot),
                      ('classifier', LinearRegression())])

LiReg_model_onehot

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier', LinearRegression())])

In [ ]:
LiReg_model_target = Pipeline(steps=[('preprocessor', preprocessor_target),
                      ('classifier', LinearRegression())])

LiReg_model_target

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('target',
                                                                   TargetEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier', LinearRegression())])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)


In [ ]:
LiReg_model_onehot.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier', LinearRegression())])

In [ ]:
LiReg_model_target.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('target',
                                                                   TargetEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier', LinearRegression())])

In [ ]:
ypred_onehot = LiReg_model_onehot.predict(X_test)
ypred_target = LiReg_model_target.predict(X_test)

In [ ]:
# Evaluating the model for onehot
mse = mean_squared_error(y_test, ypred_onehot)
r2 = r2_score(y_test, ypred_onehot)
print(f"Error values for one hot encoder:")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Error values for one hot encoder:
Mean Squared Error: 19472.41596992813
R^2 Score: 0.41391215994488595


In [ ]:
# Evaluating the model for target
mse = mean_squared_error(y_test, ypred_target)
r2 = r2_score(y_test, ypred_target)
print(f"Error values for one target encoder:")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Error values for one hot encoder:
Mean Squared Error: 19955.37802991602
R^2 Score: 0.3993757926546633


In [84]:
import numpy as np

class LinearRegressionClosedForm:
    def __init__(self):
        self.theta = None

    def fit(self, X, y):
        #column of ones to X for the intercept term
        X_b = np.c_[np.ones((X.shape[0], 1)), X]

        #Compute the closed-form solution (theta)
        self.theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

        return self

    def predict(self, X):
        if self.theta is None:
            raise ValueError("Model is not fitted yet. Please call 'fit' with appropriate data before using 'predict'.")

        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        y_pred = X_b.dot(self.theta)

        return y_pred



In [85]:

LiScratch_model_onehot = Pipeline(steps=[
    ('preprocessor', preprocessor_onehot),
    ('classifier', LinearRegressionClosedForm())  # Instantiate the model here
])


LiScratch_model_onehot


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier',
                 <__main__.LinearRegressionClosedForm object at 0x7b1ffa303b50>)])

In [86]:
LiScratch_model_target = Pipeline(steps=[
    ('preprocessor', preprocessor_target),
    ('classifier', LinearRegressionClosedForm())  # Instantiate the model here
])


LiScratch_model_target

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('target',
                                                                   TargetEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier',
                 <__main__.LinearRegressionClosedForm object at 0x7b1ffa04b9d0>)])

In [88]:
# X_train, X_test, y_train2, y_test2 = train_test_split(X, y, test_size=0.2, random_state=45)

In [92]:
LiScratch_model_onehot.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier',
                 <__main__.LinearRegressionClosedForm object at 0x7b1ffa303b50>)])

In [93]:
LiScratch_model_target.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['temp', 'atemp', 'hum',
                                                   'windspeed', 'temp_hum',
                                                   'temp_windspeed']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('target',
                                                                   TargetEncoder())]),
                                                  ['season', 'weathersit',
                                                   'day_night'])])),
                ('classifier',
                 <__main__.LinearRegressionClosedForm object at 0x7b1ffa04b9d0>)])

In [94]:
ypredls_onehot = LiScratch_model_onehot.predict(X_test)
ypredls_target = LiScratch_model_target.predict(X_test)

In [95]:
# Evaluating the model for onehot
mse = mean_squared_error(y_test, ypredls_onehot)
r2 = r2_score(y_test, ypredls_onehot)
print(f"Error values for one hot encoder:")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Error values for one hot encoder:
Mean Squared Error: 388879.6786290906
R^2 Score: -10.704641644931533


In [96]:
# Evaluating the model for target
mse = mean_squared_error(y_test, ypredls_target)
r2 = r2_score(y_test, ypredls_target)
print(f"Error values for target encoder:")
print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

Error values for target encoder:
Mean Squared Error: 19955.37802991598
R^2 Score: 0.3993757926546645
